**GOAL: Analyze the Vine DataFrame and determine if the Vine reviews are biased.**

In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Start a Spark session with the additional option that adds the driver to Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("M16_Challenge_Vine-Data-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in video games dataset from S3 Buckets
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_games_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [ ]:
# Drop null values
dropna_df = video_games_df.dropna()
dropna_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [ ]:
# Drop duplicates
clean_df = dropna_df.drop_duplicates()
clean_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   20009611|R100E0YCS06YNR|B00EFFW0HC|     277685961|       Battlefield 4|     Video Games|          5|            0|          0|   N|                Y|          Five Stars|     great thank you|2015-07-04 00:00:00|
|         US|   21036809|R100HR44BSB9I4|B007FTE2VW|     265303108|SimCity - Limited...| 

In [ ]:
# Create the Vine DataFrame
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100E0YCS06YNR|          5|            0|          0|   N|                Y|
|R100HR44BSB9I4|          1|            1|          1|   N|                Y|
|R100UABZOUF1JP|          1|            0|          0|   N|                Y|
|R101MOW8A5WC4J|          5|            0|          0|   N|                Y|
|R101X7KSBGIWF1|          1|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter the Vine DataFrame by those that had 20+ total votes
df2 = vine_df.filter(vine_df["total_votes"] >= 20)
df2.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R2SSU7RT61AYK2|          4|           87|         98|   N|                Y|
| RPRP6F6ABCLLK|          2|           28|         39|   N|                Y|
| RGOVXI764T1KO|          1|            5|         23|   N|                Y|
|R39UPDNM5TECB5|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter the Vine DataFrame by those that had 50%+ of helpful votes
df3 = df2.filter(df2["helpful_votes"]/df2["total_votes"] >= 0.5)
df3_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R2SSU7RT61AYK2|          4|           87|         98|   N|                Y|
| RPRP6F6ABCLLK|          2|           28|         39|   N|                Y|
|R39UPDNM5TECB5|          5|           23|         23|   N|                N|
|R2RW4T8KWXQXZQ|          1|           40|         65|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter the Vine DataFrame by those that have a vine review
vine_yes_df = df3.filter(df3["vine"] == "Y")
vine_yes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PU7JY8XL5M1N|          5|           19|         24|   Y|                N|
|R23WY68NT94WKE|          5|          125|        143|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
|R1UBDJF7PWB8BL|          2|           43|         59|   Y|                N|
|R1TNOMT3O5DZ0Z|          4|           27|         32|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import col, avg

In [ ]:
# Get summary statistics of those that had vine reviews
vine_yes_df.describe().show()

+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|  count|            94|                94|               94|                94|  94|               94|
|   mean|          null| 4.202127659574468|54.59574468085106|61.787234042553195|null|             null|
| stddev|          null|0.9791348741656416|65.26098459822538| 68.90976994895394|null|             null|
|    min|R10FO5UKKVZBK2|                 1|               14|                20|   Y|                N|
|    max| RYBLWBAL5M1AY|                 5|              347|               362|   Y|                N|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+



In [ ]:
# Filter the Vine DataFrame by those that *do not* have a vine review
vine_no_df = df3.filter(df3["vine"] == "N")
vine_no_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R2SSU7RT61AYK2|          4|           87|         98|   N|                Y|
| RPRP6F6ABCLLK|          2|           28|         39|   N|                Y|
|R39UPDNM5TECB5|          5|           23|         23|   N|                N|
|R2RW4T8KWXQXZQ|          1|           40|         65|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Get summary statistics of those that did not have vine reviews
vine_no_df.describe().show()

+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|  count|         40471|             40471|             40471|             40471|40471|            40471|
|   mean|          null|  3.34765634651973|47.428405524943784|55.891057794470115| null|             null|
| stddev|          null|1.6418850112078027|117.53763370687128|127.40280622961966| null|             null|
|    min|R1005PFAPGMYQS|                 1|                10|                20|    N|                N|
|    max| RZZTUIX1574RC|                 5|             10498|             10780|    N|                Y|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+



Conclusion: There could be biased among "Star Ratings". There were 40,471 records that did not have vine reviews, but only 94 records that did have vine reviews.

In [ ]:
# Get percentage of 5-star rating among those records that had vine reviews
paid_five_star_number = vines_yes_df[vine_yes_df['star_rating']== 5].count()
paid_number = vine_yes_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

94
48
0.5106382978723404


In [ ]:
# Get percentage of 5-star rating among those record that did not have vine reviews
paid_five_star_number = vine_no_df[vine_no_df['star_rating']== 5].count()
paid_number = vine_no_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

40471
15663
0.38701786464381904


Conclusion: 51% of records that were part of the Vine program gave a 5-star rating. In comparison, 38% of records that were not part of the Vine program also gave a 5-star rating. Given the fact that there were significantly less records that had Viners than those records that did have Viners, there is the possibility of biased in the Vine/Star-Rating reviews.